In [32]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [34]:
data = pd.read_csv("sentences.csv")

X = data['sentence']
y = data['lan_code']

In [35]:
vectorizer = HashingVectorizer(n_features=2**18, alternate_sign=False)

In [36]:
X_vectorized = vectorizer.transform(X)

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)

In [40]:
model = SGDClassifier(loss='log_loss', max_iter=5, tol=None)
classes = np.unique(y)

In [41]:
for batch_start in range(0, X_train.shape[0], 10000):
    X_batch = X_train[batch_start:batch_start+10000]
    y_batch = y_train.iloc[batch_start:batch_start+10000]
    if batch_start == 0:
        model.partial_fit(X_batch, y_batch, classes=classes)
    else:
        model.partial_fit(X_batch, y_batch)

In [42]:
def predict_in_batches(model, X_sparse, batch_size=10000):
    y_preds = []
    for i in range(0, X_sparse.shape[0], batch_size):
        X_batch = X_sparse[i:i+batch_size]
        preds = model.predict(X_batch)
        y_preds.extend(preds)
    return np.array(y_preds)

In [43]:
y_pred = predict_in_batches(model, X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.73


In [50]:
def predict_language(text):
    x = vectorizer.transform([text])
    return model.predict(x)[0]


print(predict_language("Bonjour, comment ça va?"))

fra
